In [10]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.signal import convolve2d
from tqdm import tqdm

from numpy import warnings
warnings.filterwarnings('ignore')

from numba import jit

# Part 1, Theory
## Question 1

In [ ]:
q1_input_signal = [4, 1, 5, 1, 4]

def up_sample(original_signal, factor):
    """
    (1) pad the original signal, with (factor - 1) zero in between each pair
    in the original signal.
    (2) performs convolution with appropriate size, to get the the linear
    interpolated up-sampling result
    """
    padded_signal = [] # start with empty, add zeros in between
    for element in original_signal:
        padded_signal.append(element)
        padded_signal.extend([0] * (factor - 1))
    # discard the last bunch of zeros
    padded_signal = np.array(padded_signal[ : -(factor - 1)])

    conv_filter = np.array([i / factor for i in list(range(0, factor)) + list(range(factor, -1, -1))])
    return np.convolve(padded_signal, conv_filter)
    
q1_upsampled = up_sample(q1_input_signal, 4)

In [ ]:
plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
ax1.set_ylabel('Signal Strength', fontsize=20)
ax1.bar(np.linspace(-2, 2, 5), q1_input_signal, width=0.04)
ax2.set_ylabel('Signal Strength', fontsize=20)
ax2.bar(np.linspace(-2, 2, len(q1_upsampled)), q1_upsampled, width=0.04)

fig.savefig("./figs/q1_upsample.pdf", bbox_inches = 'tight', pad_inches = 0)

# Part 2, Image Resizing with Seam Carving

In [ ]:
img = cv2.cvtColor(cv2.imread("./ex1.jpg"), cv2.COLOR_BGR2RGB)
grey_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(img)

In [ ]:
@jit
def calc_grad_magnitude(
    image,
    gx = np.array([
        [-1, -2, -1], [0, 0, 0], [1, 2, 1]
    ]),
    gy = np.array([
        [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]
    ])):
    """
    calculate gradient magnitudes, using gx and gy as finite difference 
    gradeint convolution operators. Default Sobel Filters.
    
    If image is 3d tensor (three color channels, then apply channel-wise)
    """
    # later figured out we should still do grey scale...
    if image.ndim == 3:
        grad_all_channels = np.zeros_like(image)
        for i in range(0, image.shape[-1]):
            sobel_x = convolve2d(image[:, :, i], gx, mode="same")
            sobel_y = convolve2d(image[:, :, i], gy, mode="same")
            grad_all_channels[:, :, i] = np.sqrt(
                sobel_x ** 2 + sobel_y ** 2
            )
        return grad_all_channels
    elif image.ndim == 2:
        return np.sqrt(convolve2d(image, gx) ** 2 + convolve2d(image, gy) ** 2)
    pass

In [ ]:
grad = calc_grad_magnitude(img)

In [ ]:
@jit
def get_energy_map(image):
    """
    bottom up DP implementation, getting the energy map from image
    (1) convolve all three channels
    (2)
    """
    R, C, channels = image.shape
    gradient_3_channel = calc_grad_magnitude(image)
    grad = cv2.cvtColor(gradient_3_channel, cv2.COLOR_RGB2GRAY)
    energy_map = np.zeros((R, C))
    # can also get this from sequencial finding min along a path, but this is easier
    track = np.zeros_like(grad, dtype=np.int)
    # all rows from second bottom row and up
    for r in range(R - 1, -1, -1):
        # each row left to right
        for c in range(C):
            # cannot vectorize assign this last row, since Numba JIT does not support such operation
            if r == R - 1: 
                energy_map[r, c] = grad[r, c]
            elif c == 0: #if left edge
                energy_map[r, c] = np.min(energy_map[r + 1, c: c + 2]) + grad[r, c]
                track[r, c] = np.argmin(energy_map[r + 1, c: c + 2]) + c - 1
            elif c == C - 1: # elif right edge
                energy_map[r, c] = np.min(energy_map[r + 1, c - 1: c + 1]) + grad[r, c]
                track[r, c] = np.argmin(energy_map[r + 1, c - 1: c + 1]) + c - 1
            else: # else center
                energy_map[r, c] = np.min(energy_map[r + 1, c - 1: c + 2]) + grad[r, c]
                track[r, c] = np.argmin(energy_map[r + 1, c - 1: c + 2]) + c - 1

    return energy_map, track

In [ ]:
emap, track = get_energy_map(img)
plt.imshow(emap)

In [ ]:
def seam_carve(image, expected_size):
    R, C, channel = image.shape
    assert channel == 3, "image received not m x n x 3"
        
    exp_R, exp_C = expected_size
    assert (R > exp_R and C == exp_C) or (R == exp_R and C > exp_C), "ERROR: can only shrink one dimension at a time! "
    
    # if more rows than expected, part2 q4 handled
    if R > exp_R:
        # rotate
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        # feed into seam carve
        image = seam_carve(image, (C, exp_R))
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE) # rotate back
        return image
    
    # now 
    for carv_idx in tqdm(range(C - exp_C)):
        mask = np.ones((R, C - carv_idx), dtype=np.bool)
        energy_map, track = get_energy_map(image)
        min_col_idx = np.argmin(energy_map[0, :])
        # the first to remove will be at (row=0, col=top_min_col_idx)
        for r in range(R): # trace all the way
            mask[r, min_col_idx] = False
            min_col_idx = track[r, min_col_idx]
            
        mask = np.stack([mask] * 3, axis=2)
        # new image with one less col
        image = image[mask].reshape((R, C - carv_idx - 1, 3))
    return image

In [ ]:
def crop_center(img, cropx, cropy):
    """
    croping center
    didn't find library function for this one, so borrowed from 
    https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image
    but I modified it so that it now supports cropping images with three color channels
    """
    y, x, _ = img.shape
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)    
    return img[starty: starty + cropy, startx: startx + cropx, :]

# part 2 q5
def compare_results(image, final_exp_size, image_title, savepath, intermediate_size=None):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))
    if intermediate_size:
        fig.suptitle(f"Image: {image_title}, from size ({image.shape[0]}, {image.shape[1]}) to {final_exp_size} via {intermediate_size}", fontsize=25)
    else:
        fig.suptitle(f"Image: {image_title}, from size ({image.shape[0]}, {image.shape[1]}) to {final_exp_size}", fontsize=25) 
    
    ax1.imshow(image)
    ax1.set_title("Original Image", fontsize=20)
    ax2.imshow(cv2.resize(image, (final_exp_size[1], final_exp_size[0])))
    ax2.set_title("Scaled Image", fontsize=20)
    ax3.imshow(crop_center(image, final_exp_size[1], final_exp_size[0]))
    ax3.set_title("Cropped Image", fontsize=20)
    # two steps !
    if intermediate_size:
        # seam carve to size 1
        carved_once = seam_carve(image, intermediate_size)
        # carve to second size
        carved = seam_carve(carved_once, final_exp_size)
    else:
        carved = seam_carve(image, final_exp_size)

    ax4.imshow(carved)
    ax4.set_title("Seam Carved Image", fontsize=20)
    fig.savefig(savepath, bbox_inches = 'tight', pad_inches = 0)
    # finally return carved result
    return carved

In [ ]:
carved = compare_results(img, (968, 957), "Castle", "./figs/castle_compare.pdf")

In [ ]:
# cloudy mountain, ex2
# (Desired sizes for seam carving: first: 961 × 1200    →   second: 861 x 1200) 
cloudy_moutain = cv2.imread("./ex2.jpg")
cloudy_moutain = cv2.cvtColor(cloudy_moutain, cv2.COLOR_BGR2RGB)
cloudy_moutain_carved = compare_results(cloudy_moutain, (861, 1200), "Cloudy Mountain", "./figs/cloudy_mountain_comp.pdf", (961, 1200))

In [ ]:
# water mountain, ex3
# (Desired sizes for seam carving:  first: 870 x 1440   →   second: 870 x 1200). 

water_mountain = cv2.imread("./ex3.jpg")
water_mountain = cv2.cvtColor(water_mountain, cv2.COLOR_BGR2RGB)
water_mountain_carved = compare_results(water_mountain, (870, 1200), "Water Mountain", "./figs/water_mountain_comp.pdf", (870, 1440))

# Part3, Corner Detection

In [11]:
# calculate the eigenvalues of the second moment matrix for each pixel I1 and I2
@jit
def calc_eigenvalues(I):
    gray_I = cv2.cvtColor(I, cv2.COLOR_RGB2GRAY)

    gray_I = gray_I * 1.1 #??
    blur = cv2.GaussianBlur(gray_I, (5, 5), 7)
    Ix = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=5)
    Iy = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=5)

    IxIy = np.multiply(Ix, Iy)
    Ix2 = np.multiply(Ix, Ix)
    Iy2 = np.multiply(Iy, Iy)

    # blurring is same as weighting, storing values at each position
    Ix2_blur = cv2.GaussianBlur(Ix2, (7, 7), 10) 
    Iy2_blur = cv2.GaussianBlur(Iy2, (7, 7), 10) 
    IxIy_blur = cv2.GaussianBlur(IxIy, (7, 7), 10)

    eigen1 = np.array([])
    eigen2 = np.array([])

    R, C = Ix2_blur.shape
    for r in tqdm(range(R)):
        for c in range(C):
            local_M = np.array([
                [Ix2_blur[r, c], IxIy_blur[r, c]],
                [IxIy_blur[r, c], Iy2_blur[r, c]]
            ])
            (e1, e2), _ = np.linalg.eig(local_M)
            np.append(eigen1, e1)
            np.append(eigen2, e2)

    return eigen1, eigen2

In [ ]:
e = calc_eigenvalues(cv2.cvtColor(cv2.imread("./part3_I1.jpg"), cv2.COLOR_BGR2RGB))



  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 1/1584 [00:00<04:13,  6.24it/s]

  0%|          | 2/1584 [00:00<04:12,  6.27it/s]

  0%|          | 3/1584 [00:00<04:10,  6.32it/s]

  0%|          | 4/1584 [00:00<04:07,  6.38it/s]

  0%|          | 5/1584 [00:00<04:07,  6.38it/s]

  0%|          | 6/1584 [00:00<04:03,  6.47it/s]

  0%|          | 7/1584 [00:01<04:02,  6.51it/s]

  1%|          | 8/1584 [00:01<04:00,  6.56it/s]

  1%|          | 9/1584 [00:01<04:00,  6.55it/s]

  1%|          | 10/1584 [00:01<04:00,  6.54it/s]

  1%|          | 11/1584 [00:01<03:58,  6.59it/s]

  1%|          | 12/1584 [00:01<03:59,  6.56it/s]

  1%|          | 13/1584 [00:02<04:01,  6.51it/s]

  1%|          | 14/1584 [00:02<04:02,  6.46it/s]

  1%|          | 15/1584 [00:02<04:02,  6.46it/s]

  1%|          | 16/1584 [00:02<04:02,  6.47it/s]

  1%|          | 17/1584 [00:02<04:03,  6.44it/s]

  1%|          | 18/1584 [00:02<04:01,  6.48it/s]

  1%|          | 19/1584 [00:02<03:58,  6.55it/

  9%|▉         | 143/1584 [00:23<04:02,  5.94it/s]

  9%|▉         | 144/1584 [00:23<04:06,  5.83it/s]

  9%|▉         | 145/1584 [00:23<04:10,  5.75it/s]

  9%|▉         | 146/1584 [00:23<04:10,  5.74it/s]

  9%|▉         | 147/1584 [00:23<04:09,  5.75it/s]

  9%|▉         | 148/1584 [00:24<04:04,  5.86it/s]

  9%|▉         | 149/1584 [00:24<03:59,  6.00it/s]

  9%|▉         | 150/1584 [00:24<03:53,  6.14it/s]

 10%|▉         | 151/1584 [00:24<03:50,  6.22it/s]

 10%|▉         | 152/1584 [00:24<03:58,  6.01it/s]

 10%|▉         | 153/1584 [00:24<04:04,  5.86it/s]

 10%|▉         | 154/1584 [00:25<04:06,  5.81it/s]

 10%|▉         | 155/1584 [00:25<04:05,  5.82it/s]

 10%|▉         | 156/1584 [00:25<04:01,  5.90it/s]

 10%|▉         | 157/1584 [00:25<04:14,  5.62it/s]

 10%|▉         | 158/1584 [00:25<04:31,  5.25it/s]

 10%|█         | 159/1584 [00:26<04:27,  5.32it/s]

 10%|█         | 160/1584 [00:26<04:21,  5.44it/s]

 10%|█         | 161/1584 [00:26<04:16,  5.54it/s]

 10%|█      

In [ ]:
e.shape